<a href="https://colab.research.google.com/github/jufabeck2202/KI-Lab/blob/main/aufgabe6/DeepReinforcementlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

General idea:
1. Use a neural network that takes as input a state (represented as numbers)
and outputs a probability for every action.
2. Generate episodes by inputing the current state into the network and
sampling actions from the network’s output. Remember the
<state, action> pairs for every episode.
3. From these episodes, identify the ones with the highest reward.
4. Use the <state, action> pairs from those high reward episodes as
training examples for improving the neural network.
5. Go back to step 2

Task 2
1. Define a neural network with two fully connected-layers. The hidden layer uses a
Relu activation. The output layer uses a softmax. Try different hidden layer sizes
(between 100 and 500). The network takes as input a vector of the current states
and gives out a probability for each action.
2. Generate 100 episodes by sampling actions using the network output. Limit the
number of steps per episode to 500. Sum up the reward of all steps of one episode.
3. Print out the mean reward per episode of the 100 episodes.
4. Identify the 20 best of those episodes in terms of reward and use the
<state, action> pairs of these episodes as training examples for the network.
5. Update the weights of the network by performing backpropagation on these <state,
action> pairs.
6. Repeat steps 2 to 5 until a mean reward of 100 is reached.
7. Record a video of the lunar lander by running the trained network on one additional
episode.

Hints
* Use !pip3 install box2d-py to make the environment work in Colab.
* You cannot show the video of your lander in Colab (env.render() fails).  
* Workaround: Download the model on your local machine and record the video there,
using recording_demo.py as template (see Mattermost).
* The loss is not a useful indicator for the learning progress in RL. Instead check how
the mean reward develops over time.
* The mean reward will jump back and forth quite a bit, but overall should increase.
* After roughly 70 training iterations the mean reward should be positive, and after
roughly 100 steps be over 100.
* Note that these numbers depend on your parameter setting and it may also take
longer or shorter.
* Reinforcement learning is much more difficult than supervised learning, you have to
play around quite a bit to get things into the right direction.

# Imports

In [1]:
!pip3 install box2d-py gym > /dev/null

In [2]:
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
import numpy as np
import matplotlib.pyplot as plt
import heapq
import random
import torch.optim as optim
%matplotlib inline


In [3]:
env = gym.make('LunarLander-v2')
rewards = []

##get Action space
print(env.observation_space.shape)
print(env.action_space)
print()
print(env.observation_space.sample())
print(env.action_space.sample())


(8,)
Discrete(4)

[ 0.57051045 -1.5992639   0.59137917  0.51901096 -1.4794146   0.02208187
  0.24469182  0.5068234 ]
0


# Define Network

In [4]:

class Network(nn.Module):
    def __init__(self, hidden_size,):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(env.observation_space.shape[0], hidden_size)
        self.fc2 = nn.Linear(hidden_size, env.action_space.n)
        
    def forward(self, x):
        x = f.relu(self.fc1(x))
        return self.fc2(x)

In [5]:
torch.backends.cudnn.enabled = True
GPU_ON = torch.cuda.is_available()
device = torch.device("cuda:0" if GPU_ON else "cpu")
net = Network(200)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
if GPU_ON:
  net.cuda()

# Sample Episodes

In [6]:
def sample(episodes_n=100, max_steps=5000):
  episodes_data = []
  softmax = nn.Softmax(dim=1)
  for i_episode in range(episodes_n):
      steps = []
      rewards = []
      state = env.reset()
      for t in range(max_steps):

          state_vector = torch.FloatTensor([state])
          probability= softmax(net(state_vector))
          selected_action = np.random.choice(len(probability.data.numpy()[0]), p=probability.data.numpy()[0])
          new_state, reward, done, info = env.step(selected_action)
          rewards.append(reward)
          #imporant: use old state
          steps.append([selected_action, state])
          state = new_state
          

          if done:
              episodes_data.append({ "mean_reward": np.array(rewards).sum() ,"steps": steps})
              break
      #print(f"Mean reward of episode {i_episode}: {np.array(rewards).mean()}")
  return np.array(episodes_data)

In [7]:
t = sample()


In [8]:
def calculate_total_mean_reward(all_episodes):
   all_rewards = [d['mean_reward'] for d in all_episodes]
   return np.array(all_rewards).mean()

In [9]:
def get_top_20(all_episodes):
  return heapq.nlargest(20, all_episodes, key=lambda s: s['mean_reward'])

In [10]:
def get_training_data(episode_data):
  train_data = []
  for entry in episodes_data:
    for step in entry['steps']:
      one_hot = np.zeros(env.action_space.n)
      one_hot[step[0]] = 1
      train_data.append([one_hot, step[1]])
  return train_data


In [11]:
def get_training_data_batch(episode_data):
  x_data = []
  y_data = []
  for entry in episodes_data:
    for step in entry['steps']:
      #state
      x_data.append(step[1])
      #action
      y_data.append(step[0])

  return [x_data, y_data]

In [12]:
def train(training_data):
  for data in training_data:
    optimizer.zero_grad()
    # get the inputs; data is a list of [inputs, labels]
    action, observation = data
    #convert to torch?
    observation=torch.from_numpy(np.expand_dims(observation, axis=0))
    action =torch.from_numpy(np.array([np.argmax(action, axis=0)]))
    if GPU_ON:
      action = action.cuda()
      observation = observation.cuda()

    # forward + backward + optimize
    output_action = net(observation)
    loss = criterion(output_action, action)
    loss.backward()
    optimizer.step()

def train_batch(batch_data):
  optimizer.zero_grad()
  # get the inputs; data is a list of [inputs, labels]
  x_data, y_data = batch_data
  #convert to torch?
  observation=torch.FloatTensor(x_data)
  target =torch.LongTensor(y_data)
  
  if GPU_ON:
    target = target.cuda()
    observation = observation.cuda()

  # forward + backward + optimize
  action_pred = net(observation)
  #print(action_pred[121])
  #print(action_pred[-1])
  #print(target[121])
  #print(target[-1])
  loss = criterion(action_pred, target)
  print(f"Loss {loss}")
  loss.backward()
  optimizer.step()



In [13]:
#train_batch(get_training_data_batch(best_20))

In [14]:
while False:
  episodes_data = sample(episodes_n=200)
  print(f"Mean Reward for Sampling 20 episodes {calculate_total_mean_reward(episodes_data)}")
  best_20 = get_top_20(episodes_data)
  train(get_training_data(best_20))


  

In [ ]:
for i in range(10000):
  episodes_data = sample()
  print(f"{i} Mean Reward for Sampling 20 episodes {calculate_total_mean_reward(episodes_data)}")
  best_20 = get_top_20(episodes_data)
  train_batch(get_training_data_batch(best_20))

0 Mean Reward for Sampling 20 episodes -237.00387442808847
Loss 1.3673635721206665
1 Mean Reward for Sampling 20 episodes -260.7828109804145
Loss 1.3259042501449585
2 Mean Reward for Sampling 20 episodes -376.7105754603966
Loss 1.175499439239502
3 Mean Reward for Sampling 20 episodes -427.1510052189414
Loss 1.005089282989502
4 Mean Reward for Sampling 20 episodes -485.2583841271655
Loss 0.8276054263114929
5 Mean Reward for Sampling 20 episodes -515.7420076757255
Loss 0.5990819334983826
6 Mean Reward for Sampling 20 episodes -521.0502960743398
Loss 0.45845773816108704
7 Mean Reward for Sampling 20 episodes -548.3841626790886
Loss 0.3504909574985504
8 Mean Reward for Sampling 20 episodes -562.5673020089976
Loss 0.2851387560367584
9 Mean Reward for Sampling 20 episodes -583.9915371240138
Loss 0.23019196093082428
10 Mean Reward for Sampling 20 episodes -603.9455182964574
Loss 0.1900959312915802
11 Mean Reward for Sampling 20 episodes -550.7643337560994
Loss 0.17945270240306854
12 Mean Rewa